## Interacting with external data -- our own pre-workshop survey

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
surveylink = 'https://docs.google.com/spreadsheets/d/1n3JBKM_Wp7TcJDPl05bR6joVWbFbmLiVkRbwUU-IhXg/export?format=csv'

In [ ]:
df = pd.read_csv(surveylink)

In [ ]:
df

In [ ]:
for row in df.index:
    print(df.iloc[row][1:].values)

In [ ]:
connectionPreworkshop = sqlite3.connect('preworkshop.db')

In [ ]:
cursorPreworkshop = connectionPreworkshop.cursor()

In [ ]:
cursorPreworkshop.execute('''
CREATE TABLE preworkshop (sql text,
                        python text,
                        jupyter text,
                        pandas text);
''')

In [ ]:
for row in df.index:
    answers = df.iloc[row][1:].values
    cursorPreworkshop.execute('''INSERT INTO preworkshop (sql, python, jupyter, pandas) 
            VALUES (?,?,?,?);''', answers)

In [ ]:
connectionPreworkshop.commit()

In [ ]:
df = pd.read_sql_query('select * from preworkshop',connectionPreworkshop)

In [ ]:
df

In [ ]:
df['pandas'].value_counts().plot.barh()

In [ ]:
cursorPreworkshop.execute('''
CREATE TABLE answers (answerText text,
                        answerCode integer);
''')

In [ ]:
answers = [
    ['Use it all the time',4],
    ['Use them all the time',4],
    ['Not an expert, but comfortable with reading commands',3],
    ['Not an expert, but comfortable with running them',3],
    ['Familiar with it but could use a refresher',2],
    ['Familiar with them but could use a refresher',2],
    ['Not at all',1],
    ['What\'s Pandas?',0],
]

for i in answers:
    cursorPreworkshop.execute('''INSERT INTO answers (answerText, answerCode) 
            VALUES (?,?);''', i)

In [ ]:
connectionPreworkshop.commit()

In [ ]:
df = pd.read_sql_query('select * from answers',connectionPreworkshop)

In [ ]:
df

In [ ]:
df = pd.read_sql_query('''
select a1.answerCode as sql, 
    a2.answerCode as python, 
    a3.answerCode as jupyter, 
    a4.answerCode as pandas,
    (a1.answerCode + a2.answerCode + a3.answerCode + a4.answerCode)/16. as Mastery
from preworkshop p
join answers a1
on p.sql = a1.answerText
join answers a2
on p.python = a2.answerText
join answers a3
on p.jupyter = a3.answerText
join answers a4
on p.pandas = a4.answerText
''',connectionPreworkshop)

In [ ]:
df

In [ ]:
df['Mastery'].plot.hist(range=(0.0,1.0))